In [1]:
import arrow
import numpy as np
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import joblib 
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import TensorDataset
from sklearn.metrics import confusion_matrix

from models.SimpleAutoEncoder import SimpleAutoEncoder

from utils.evalUtils import calc_cm_metrics
from utils.evalUtils import print_confusion_matrix

In [2]:
from matplotlib import rc
rc('text', usetex=True)

In [3]:
%run -i ./scripts/setConfigs.py

Set configs..


In [4]:
os.getcwd()

'/home/torge/dev/masterthesis_code/02_Experimente/03_Experimente'

In [5]:
%run -i ./scripts/EvalPreperations.py

Set configs..
Read the data..
Shape of normal data (X_sim): (105216, 17)
Shape of anormal data (X_test): (35040, 18)
Shape of drifted data (X_drifted): (35040, 18)
Shape of drifted anormal data (X_drifted,anormal): (35040, 19)
Save label..
Shape of anormal data (X_test): (35040, 17)
Shape of drifted data (X_drifted): (35040, 17)
Shape of drifted anormal data (X_drifted,anormal): (35040, 17)
Scale data..
Prepare data for PyTorch..


In [6]:
type(drifted_anormal_torch_tensor)

torch.Tensor

In [ ]:
os.chdir(os.path.join(exp_data_path, 'experiment'))
extension = 'csv'
result = glob.glob('*.{}'.format(extension))

In [24]:
path = os.path.join(exp_data_path, 'experiment', 'fine_tuning', '*.csv')
result = glob.glob(path)

In [41]:
real_results = []
for r in result:
    if 'tVPII_M2' in r:
        real_results.append(r)

In [42]:
len(real_results)

37

In [43]:
df_tVP2_m2 = pd.read_csv(real_results[0], sep=';')

In [44]:
df_tVP2_m2.head()

,optimizer,fine_tune_classes,name_pretrained_model,k,fine_tune_iterations,lr,model_fn,pretrained_model_fn,logreg_fn,TP_x_test,TN_x_test,FP_x_test,FN_x_test,TP_x_drifted_ano,TN_x_drifted_ano,FP_x_drifted_ano,FN_x_drifted_ano,Accuracy_x_test,Precision_x_test,Specifity_x_test,Sensitivity_x_test,Accuracy_x_drifted_ano,Precision_x_drifted_ano,Specifity_x_drifted_ano,Sensitivity_x_drifted_ano
0,Adam,"[2, 3]",M2,5,1,0.1,/home/torge/dev/masterthesis_code/02_Experimen...,20200303_LogRegModell.save,/home/torge/dev/masterthesis_code/02_Experimen...,2496,4831,27712,1,2494,4339,28204,3,20.910388,8.262712,65.934216,99.959952,19.500571,8.124308,63.500659,99.879856


In [45]:
for file in real_results[1:]:
    df = pd.read_csv(file, sep=';')
    df_tVP2_m2 = df_tVP2_m2.append(df)

In [46]:
df_tVP2_m2.reset_index(inplace=True, drop=True)

In [50]:
df_tVP2_m2.head()

,optimizer,fine_tune_classes,name_pretrained_model,k,fine_tune_iterations,lr,model_fn,pretrained_model_fn,logreg_fn,TP_x_test,TN_x_test,FP_x_test,FN_x_test,TP_x_drifted_ano,TN_x_drifted_ano,FP_x_drifted_ano,FN_x_drifted_ano,Accuracy_x_test,Precision_x_test,Specifity_x_test,Sensitivity_x_test,Accuracy_x_drifted_ano,Precision_x_drifted_ano,Specifity_x_drifted_ano,Sensitivity_x_drifted_ano
0,Adam,"[2, 3]",M2,5,1,0.10,/home/torge/dev/masterthesis_code/02_Experimen...,20200303_LogRegModell.save,/home/torge/dev/masterthesis_code/02_Experimen...,2496,4831,27712,1,2494,4339,28204,3,20.910388,8.262712,65.934216,99.959952,19.500571,8.124308,63.500659,99.879856
1,SGD,[1],M2,5,1,0.01,/home/torge/dev/masterthesis_code/02_Experimen...,20200303_LogRegModell.save,/home/torge/dev/masterthesis_code/02_Experimen...,2486,7984,24559,11,2490,7042,25501,7,29.880137,9.192087,76.255969,99.559471,27.203196,8.895716,73.877465,99.719664
2,Adam,[1],M2,10,64,0.10,/home/torge/dev/masterthesis_code/02_Experimen...,20200303_LogRegModell.save,/home/torge/dev/masterthesis_code/02_Experimen...,2494,1005,31538,3,2497,962,31581,0,9.985731,7.328397,28.722492,99.879856,9.871575,7.327308,27.811506,100.000000
3,SGD,"[1, 3]",M2,5,1,0.10,/home/torge/dev/masterthesis_code/02_Experimen...,20200303_LogRegModell.save,/home/torge/dev/masterthesis_code/02_Experimen...,2400,32543,0,97,2394,29230,3313,103,99.723174,100.000000,93.131672,96.115338,90.251142,41.948484,92.429800,95.875050
4,Adam,[2],M2,20,64,0.10,/home/torge/dev/masterthesis_code/02_Experimen...,20200303_LogRegModell.save,/home/torge/dev/masterthesis_code/02_Experimen...,2485,15295,17248,12,2485,13652,18891,12,50.742009,12.593118,86.023622,99.519423,46.053082,11.625187,84.600607,99.519423


In [48]:
df_results_tVP2_m2 = pd.DataFrame()

In [49]:
df_results_tVP2_m2['ano_labels'] = s_ano_labels_drifted_ano
df_results_tVP2_m2.reset_index(inplace=True, drop=True)

In [54]:
for i, row in df_tVP2_m2.iterrows():
    print('Current Iteration: {} of {}'.format(i+1, len(df_tVP2_m2)))
    model = None
    model = SimpleAutoEncoder(num_inputs=17, val_lambda=42)
    logreg = None
    
    model_fn = row['model_fn']
    logreg_fn = row['logreg_fn']
    
    logreg = joblib.load(logreg_fn)
    model.load_state_dict(torch.load(model_fn))
    
    losses = []
    for val in drifted_anormal_torch_tensor:
        loss = model.calc_reconstruction_error(val)
        losses.append(loss.item())
    
    s_losses_anormal = pd.Series(losses)
    
    X = s_losses_anormal.to_numpy()
    X = X.reshape(-1, 1)
    
    predictions = []
    for val in X:
        val = val.reshape(1,-1)
        pred = logreg.predict(val)
        predictions.append(pred[0])

    col_name = 'preds_model_{}'.format(i)
    
    df_results_tVP2_m2[col_name] = predictions

Current Iteration: 1 of 37


/home/torge/anaconda3/envs/ma_modells/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Current Iteration: 2 of 37


/home/torge/anaconda3/envs/ma_modells/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Current Iteration: 3 of 37


...

/home/torge/anaconda3/envs/ma_modells/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Current Iteration: 36 of 37


/home/torge/anaconda3/envs/ma_modells/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Current Iteration: 37 of 37


/home/torge/anaconda3/envs/ma_modells/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [55]:
file_name = '{}_predictions_models_m2_phase_2_on_x_drifted_ano.csv'.format(arrow.now().format('YYYYMMDD'))
full_fn = '/home/torge/dev/masterthesis_code/02_Experimente/03_Experimente/exp_data/evaluation/{}'.format(file_name)
df_results_tVP2_m2.to_csv(full_fn, sep=';', index=False)

In [56]:
len(df_results_tVP2_m2.columns)

38

In [57]:
len(df_results_tVP2_m2)

35040

In [58]:
df_results_tVP2_m2.describe()

,ano_labels,preds_model_0,preds_model_1,preds_model_2,preds_model_3,preds_model_4,preds_model_5,preds_model_6,preds_model_7,preds_model_8,preds_model_9,preds_model_10,preds_model_11,preds_model_12,preds_model_13,preds_model_14,preds_model_15,preds_model_16,preds_model_17,preds_model_18,preds_model_19,preds_model_20,preds_model_21,preds_model_22,preds_model_23,preds_model_24,preds_model_25,preds_model_26,preds_model_27,preds_model_28,preds_model_29,preds_model_30,preds_model_31,preds_model_32,preds_model_33,preds_model_34,preds_model_35,preds_model_36
count,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000,35040.000000
mean,0.143208,0.876084,0.798830,0.972546,0.162871,0.610046,0.798744,0.764840,0.723059,0.820548,0.758676,0.164412,0.757449,0.801341,0.760873,0.727340,0.707648,0.819349,0.909132,0.900342,0.162529,0.667608,0.789384,0.720120,0.970862,0.730736,0.163385,0.163413,0.730736,0.164269,0.799458,0.185788,0.757877,0.184047,0.725685,0.733647,0.164269,0.900400
std,0.518759,0.329490,0.400881,0.163406,0.369253,0.487747,0.400944,0.424105,0.447493,0.383736,0.427893,0.370654,0.428632,0.398996,0.426556,0.445334,0.454849,0.384734,0.287425,0.299547,0.368940,0.471077,0.407752,0.448947,0.168196,0.443584,0.369722,0.369748,0.443584,0.370525,0.400412,0.388941,0.428375,0.387528,0.446175,0.442057,0.370525,0.299471
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000
75%,0.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000
max,3.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
